# SUDOKU

In [3]:
import random
import svgwrite
from PIL import Image, ImageDraw, ImageFont

def generar_sudoku():
    # Crear una matriz vacía de 9x9
    matriz = [[0] * 9 for _ in range(9)]

    # Rellenar la matriz con números válidos
    resolver_sudoku(matriz)

    # Quitar algunos números para crear espacios en blanco
    quitar_numeros(matriz)

    return matriz

def resolver_sudoku(matriz):
    # Encontrar la próxima celda vacía
    fila, columna = encontrar_celda_vacia(matriz)

    # Si no hay más celdas vacías, el sudoku está resuelto
    if fila is None:
        return True

    # Obtener una lista aleatoria de números del 1 al 9
    numeros = list(range(1, 10))
    random.shuffle(numeros)

    # Probar los números en orden aleatorio
    for num in numeros:
        if validar_numero(matriz, fila, columna, num):
            matriz[fila][columna] = num

            # Intentar resolver el sudoku recursivamente
            if resolver_sudoku(matriz):
                return True

            # Si no se puede resolver, retroceder y probar otro número
            matriz[fila][columna] = 0

    # Si no se encuentra ninguna solución, regresar False
    return False


def encontrar_celda_vacia(matriz):
    # Encontrar la primera celda vacía en la matriz
    for fila in range(9):
        for columna in range(9):
            if matriz[fila][columna] == 0:
                return fila, columna
    return None, None

def validar_numero(matriz, fila, columna, num):
    # Verificar si el número es válido en la fila
    if num in matriz[fila]:
        return False

    # Verificar si el número es válido en la columna
    for i in range(9):
        if matriz[i][columna] == num:
            return False

    # Verificar si el número es válido en el bloque 3x3
    bloque_fila = fila // 3
    bloque_columna = columna // 3

    for i in range(bloque_fila * 3, bloque_fila * 3 + 3):
        for j in range(bloque_columna * 3, bloque_columna * 3 + 3):
            if matriz[i][j] == num:
                return False

    return True

def quitar_numeros(matriz):
    # Determinar la cantidad de números a quitar (aproximadamente)
    espacios_vacios = random.randint(45, 55)

    for _ in range(espacios_vacios):
        # Elegir una celda aleatoria
        fila = random.randint(0, 8)
        columna = random.randint(0, 8)

        # Si la celda ya está vacía, elegir otra celda
        while matriz[fila][columna] == 0:
            fila = random.randint(0, 8)
            columna = random.randint(0, 8)

        # Quitar el número de la celda
        matriz[fila][columna] = 0

def imprimir_sudoku(matriz):
    for fila in matriz:
        print(' '.join(str(num) for num in fila))

def imprimir_sudoku_en_png(sudoku, ruta_archivo):
    FILAS = len(sudoku)
    COLUMNAS = len(sudoku[0])
    ANCHO_CELDA = 60
    ALTO_CELDA = 60
    ANCHO_IMAGEN = COLUMNAS * ANCHO_CELDA
    ALTO_IMAGEN = FILAS * ALTO_CELDA

    imagen = Image.new("RGB", (ANCHO_IMAGEN, ALTO_IMAGEN), "white")
    dibujo = ImageDraw.Draw(imagen)
    fuente = ImageFont.truetype("./arial/arial.ttf", 24)

    # Dibujar las líneas divisorias verticales
    for columna in range(1, COLUMNAS):
        x = columna * ANCHO_CELDA
        ancho_linea = 2 if columna % 3 != 0 else 8
        dibujo.line([(x, 0), (x, ALTO_IMAGEN)], fill="black", width=ancho_linea)

    # Dibujar las líneas divisorias horizontales
    for fila in range(1, FILAS):
        y = fila * ALTO_CELDA
        ancho_linea = 2 if fila % 3 != 0 else 8
        dibujo.line([(0, y), (ANCHO_IMAGEN, y)], fill="black", width=ancho_linea)

    # Rellenar los números en las celdas
    for fila in range(FILAS):
        for columna in range(COLUMNAS):
            numero = sudoku[fila][columna]
            if numero != 0:
                x = columna * ANCHO_CELDA + ANCHO_CELDA // 2
                y = fila * ALTO_CELDA + ALTO_CELDA // 2
                dibujo.text((x, y), str(numero), font=fuente, fill="black", anchor="mm")
    
    # Dibujar el contorno de la imagen
    dibujo.rectangle([(0, 0), (ANCHO_IMAGEN - 1, ALTO_IMAGEN - 1)], outline="black", width=2)

    # Guardar la imagen en el archivo PNG
    imagen.save(ruta_archivo, "PNG")    

def imprimir_sudoku_en_svg(sudoku, ruta_archivo):
    FILAS = len(sudoku)
    COLUMNAS = len(sudoku[0])
    ANCHO_CELDA = 60
    ALTO_CELDA = 60
    ANCHO_IMAGEN = COLUMNAS * ANCHO_CELDA
    ALTO_IMAGEN = FILAS * ALTO_CELDA

    dwg = svgwrite.Drawing(ruta_archivo, profile='tiny')
    dwg.viewbox(0, 0, ANCHO_IMAGEN, ALTO_IMAGEN)
    dwg.add(dwg.rect((0, 0), (ANCHO_IMAGEN, ALTO_IMAGEN), fill='white', stroke='black', stroke_width=2))

    # Dibujar las líneas divisorias verticales
    for columna in range(1, COLUMNAS):
        x = columna * ANCHO_CELDA
        ancho_linea = 2 if columna % 3 != 0 else 8
        dwg.add(dwg.line((x, 0), (x, ALTO_IMAGEN), stroke='black', stroke_width=ancho_linea))

    # Dibujar las líneas divisorias horizontales
    for fila in range(1, FILAS):
        y = fila * ALTO_CELDA
        ancho_linea = 2 if fila % 3 != 0 else 8
        dwg.add(dwg.line((0, y), (ANCHO_IMAGEN, y), stroke='black', stroke_width=ancho_linea))

    # Rellenar los números en las celdas
    for fila in range(FILAS):
        for columna in range(COLUMNAS):
            numero = sudoku[fila][columna]
            if numero != 0:
                x = columna * ANCHO_CELDA + ANCHO_CELDA // 2
                y = fila * ALTO_CELDA + ALTO_CELDA // 2 + 10 
                dwg.add(dwg.text(str(numero), insert=(x, y), font_size=24, text_anchor='middle'))
    dwg.save()


## Generar Sudoku

In [4]:
# Generar y mostrar un sudoku
sudoku = generar_sudoku()
imprimir_sudoku(sudoku)
imprimir_sudoku_en_svg(sudoku,"./sudoku.svg")

0 0 0 0 8 6 0 0 0
0 0 4 1 0 0 0 5 0
6 0 7 3 0 0 0 0 0
2 6 8 0 5 1 0 9 0
7 4 9 8 6 3 0 1 5
0 1 0 7 9 0 0 0 6
0 0 3 0 0 4 0 0 0
0 0 6 5 1 0 0 0 0
0 7 0 0 3 9 0 6 0


## Crear 10 Sudokus aleatorios

In [5]:
for i in range(11):
    sudoku = generar_sudoku()
    imprimir_sudoku_en_svg(sudoku,"./sudokus_generados/para_resolver/sudoku"+str(i)+".svg")
    mensaje = ("Sudoku: " + str(i) + " generado") if resolver_sudoku(sudoku) else "Error sudoku " +str(i) + " no se pudo resolver."
    print(mensaje)
    imprimir_sudoku_en_svg(sudoku,"./sudokus_generados/soluciones/sudoku"+str(i)+".svg")

FileNotFoundError: [Errno 2] No such file or directory: './sudokus_generados/para_resolver/sudoku0.svg'

In [6]:
from reportlab.lib.pagesizes import letter
from reportlab.pdfgen import canvas

def imprimir_sudokus_en_pdf(sudokus,dim_cuadricula, numero_pagina, bloque_sudoku, c):
    NUM_SUDOKUS = len(sudokus)
    SUDOKUS_POR_FILA = dim_cuadricula
    SUDOKUS_POR_COLUMNA = dim_cuadricula
    
    TOTAL_FILAS = NUM_SUDOKUS // SUDOKUS_POR_FILA

    ANCHO_PAGINA, ALTO_PAGINA = letter

    MARGEN_IZQUIERDO = 36
    MARGEN_INFERIOR = 46
    
    ESPACIO_INTERSUDOKU = 16

    ANCHO_SUDOKU = (ANCHO_PAGINA - 2 * MARGEN_IZQUIERDO - (SUDOKUS_POR_FILA - 1) * ESPACIO_INTERSUDOKU) / SUDOKUS_POR_FILA
    ALTO_SUDOKU = (ALTO_PAGINA - 2 * MARGEN_INFERIOR - (SUDOKUS_POR_COLUMNA - 1) * ESPACIO_INTERSUDOKU) / SUDOKUS_POR_COLUMNA    

    for i in range(NUM_SUDOKUS):
        fila = i // SUDOKUS_POR_FILA
        columna = i % SUDOKUS_POR_FILA

        sudoku = sudokus[i]
        
        FILAS = len(sudoku)
        COLUMNAS = len(sudoku[0])

        ANCHO_CELDA = ANCHO_SUDOKU / COLUMNAS
        ALTO_CELDA = ALTO_SUDOKU / FILAS

        x_inicio = MARGEN_IZQUIERDO + columna * (ANCHO_SUDOKU + ESPACIO_INTERSUDOKU)
        y_inicio = MARGEN_INFERIOR + (TOTAL_FILAS-1-fila) * (ALTO_SUDOKU + ESPACIO_INTERSUDOKU)
        
        # Dibujar el borde exterior del sudoku
        c.setLineWidth(2)
        c.rect(x_inicio, y_inicio, ANCHO_SUDOKU, ALTO_SUDOKU)

        # Dibujar las líneas divisorias verticales
        for line in range(1, COLUMNAS):
            x = x_inicio + line * ANCHO_CELDA
            if line % 3 != 0:
                c.setLineWidth(1)
            else:
                c.setLineWidth(2)            
            c.line(x, y_inicio, x, y_inicio + ALTO_SUDOKU)

        # Dibujar las líneas divisorias horizontales
        for line in range(1, FILAS):
            y = y_inicio + line * ALTO_CELDA
            if line % 3 != 0:
                c.setLineWidth(1)
            else:
                c.setLineWidth(2)                
            c.line(x_inicio, y, x_inicio + ANCHO_SUDOKU, y)
        
        # Número sudoku
        c.setFont("Helvetica", 3*(ESPACIO_INTERSUDOKU//4))
        c.drawCentredString(x_inicio + ANCHO_SUDOKU/2, y_inicio + ALTO_SUDOKU + ESPACIO_INTERSUDOKU//4, str(bloque_sudoku + i) )
        
        # Rellenar los números en las celdas
        for f in range(FILAS):
            for col in range(COLUMNAS):
                numero = sudoku[f][col]
                if numero != 0:
                    x = x_inicio + col * ANCHO_CELDA + ANCHO_CELDA / 2
                    y = y_inicio + f * ALTO_CELDA + ALTO_CELDA / 2
                    c.setFont("Helvetica", 12)
                    c.drawCentredString(x, y, str(numero))
                    
        # Agregar número de página al centro inferior de la página
        c.setFont("Helvetica", 10)
        c.drawCentredString(ANCHO_PAGINA / 2, MARGEN_INFERIOR - 20, f"Página {numero_pagina}")



In [11]:
from PyPDF4 import PdfFileReader, PdfFileMerger
import math

def calcular_mcm(num1, num2):
    # Calcula el máximo común divisor (MCD) utilizando el algoritmo de Euclides
    def calcular_mcd(a, b):
        while b != 0:
            a, b = b, a % b
        return a

    # Calcula el MCM utilizando la fórmula MCM = (num1 * num2) / MCD(num1, num2)
    mcd = calcular_mcd(num1, num2)
    mcm = (num1 * num2) // mcd

    return mcm

def generar_libro_sudokus_pdf(nombre_libro,dim_cuadricula_ejercicios,dim_cuadricula_soluciones, min_paginas, desfase_paginas):
    ruta_archivo_sudokus = nombre_libro + "Vacios.pdf"
    ruta_archivo_soluciones = nombre_libro + "Soluciones.pdf"
    canvas_sudokus = canvas.Canvas(ruta_archivo_sudokus, pagesize=letter)
    canvas_soluciones = canvas.Canvas(ruta_archivo_soluciones, pagesize=letter)
            
    ejercicios_pagina = dim_cuadricula_ejercicios**2
    soluciones_pagina = dim_cuadricula_soluciones**2
    batch_size = calcular_mcm(ejercicios_pagina ,soluciones_pagina)
    #math.gcd(ejercicios_pagina ,soluciones_pagina)

    total_batchs = math.ceil(min_paginas * ejercicios_pagina / batch_size)
    
    pagina = 1 + desfase_paginas
    pagina_sol = 1 + desfase_paginas + math.ceil(min_paginas * ejercicios_pagina / batch_size)*(batch_size/ejercicios_pagina)
    
    print(batch_size)
    print(ejercicios_pagina)
    print(soluciones_pagina)
    print(total_batchs)
    
    for batch in range(total_batchs):
        matriz_ejercicios = [generar_sudoku() for _ in range(batch_size)]        
        print("IMPRIMIENDO EJERCICIOS:")
        for i in range(int(batch_size/ejercicios_pagina)):
            bloque_sudoku = i*ejercicios_pagina+batch_size*batch + 1
            imprimir_sudokus_en_pdf(matriz_ejercicios[i*ejercicios_pagina:i*ejercicios_pagina+ejercicios_pagina], dim_cuadricula_ejercicios, pagina, bloque_sudoku, canvas_sudokus)                        
            a= str(i*ejercicios_pagina)
            b= str(i*ejercicios_pagina+ejercicios_pagina)
            print(a+":"+b)
            pagina = pagina + 1
            canvas_sudokus.showPage()        

        for sudoku in matriz_ejercicios:
            resolver_sudoku(sudoku)    
            
        print("IMPRIMIENDO SOLUCIONES:")
        for i in range(int(batch_size/soluciones_pagina)):
            bloque_sudoku = i*soluciones_pagina+batch_size*batch + 1
            imprimir_sudokus_en_pdf(matriz_ejercicios[i*soluciones_pagina:i*soluciones_pagina+soluciones_pagina], dim_cuadricula_soluciones, pagina_sol, bloque_sudoku, canvas_soluciones)
            print(str(i*soluciones_pagina)+":"+str(i*soluciones_pagina+soluciones_pagina))
            pagina_sol = pagina_sol + 1
            canvas_soluciones.showPage()        
        
    """""
    for i in range(total_paginas):
        matriz_sudokus = [generar_sudoku() for _ in range(dim_cuadricula * dim_cuadricula)]
        imprimir_sudokus_en_pdf(matriz_sudokus, dim_cuadricula, desfase_paginas + i, canvas_sudokus)
        canvas_sudokus.showPage()

        for sudoku in matriz_sudokus:
            resolver_sudoku(sudoku)

        imprimir_sudokus_en_pdf(matriz_sudokus, dim_cuadricula, desfase_paginas + total_paginas + i, canvas_soluciones)
        canvas_soluciones.showPage()
    """

    canvas_sudokus.save()
    canvas_soluciones.save()

    # Crear el objeto PdfFileMerger
    merger = PdfFileMerger()

    # Agregar los archivos PDF al objeto PdfFileMerger
    merger.append(PdfFileReader(ruta_archivo_sudokus, "rb"))
    merger.append(PdfFileReader(ruta_archivo_soluciones, "rb"))

    # Unir los archivos en uno solo
    merger.write(nombre_libro + "Sudokus.pdf")
    merger.close()

    print("Terminado")

In [12]:
# Ejemplo de uso
dim_cuadricula_ejercicios = 2
dim_cuadricula_soluciones = 3
min_paginas = 10
desfase_paginas = 0
nombre_libro = "./sudokus_generados/Libro_2X2_"
generar_libro_sudokus_pdf(nombre_libro,dim_cuadricula_ejercicios,dim_cuadricula_soluciones, min_paginas, desfase_paginas)

36
4
9
2
IMPRIMIENDO EJERCICIOS:
0:4
4:8
8:12
12:16
16:20
20:24
24:28
28:32
32:36
IMPRIMIENDO SOLUCIONES:
0:9
9:18
18:27
27:36
IMPRIMIENDO EJERCICIOS:
0:4
4:8
8:12
12:16
16:20
20:24
24:28
28:32
32:36
IMPRIMIENDO SOLUCIONES:
0:9
9:18
18:27
27:36
Terminado
